In [1]:
from collections import namedtuple
import numpy as np
import nest
from scaffold.core import from_hdf5
from scaffold.config import JSONConfig
from scaffold.output import HDF5Formatter
from population_view import PopView
import world
from world_populations import Planner, Cortex, SensoryIO, MotorIO, DirectDCN, InverseDCN

In [2]:
Cerebellum = namedtuple("Cerebellum", "mf io dcn")
Brain = namedtuple("Brain", "planner cortex forward inverse")

In [3]:
CORES = 8

nest.Install("extracerebmodule")

nest.set_verbosity('M_WARNING')
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads' : CORES,
                      'total_num_virtual_procs' : CORES,
                      'resolution' : 1.0,
                      'overwrite_files' : True})

In [4]:
trial_len = 300

In [5]:
def create_cortex(MF_number, prism):
    planner_pv = Planner(MF_number, prism)
    cortex_pv = Cortex(MF_number)
    
    planner_pv.connect(cortex_pv)
    
    return planner_pv, cortex_pv

In [6]:
def create_brain(prism):
    hdf5_file = 'mouse_cerebellum_200.hdf5'
    # hdf5_file = 'scaffold_network_VOR.hdf5'
    # json_file = 'scaffold_network_VOR_10.json'
    
    # reconfigured_obj = JSONConfig(json_file)
    # HDF5Formatter.reconfigure(hdf5_file, reconfigured_obj)
    
    scaffold_model = from_hdf5(hdf5_file)
    scaffold_model.configuration.verbosity = 3

    # Get scaffold model populations
    S_MF = scaffold_model.get_entities_by_type("mossy_fibers")
    S_IO = scaffold_model.get_cells_by_type("io_cell")[:, 0]
    S_DCN = scaffold_model.get_cells_by_type("dcn_cell")[:, 0]

    uz_pos = scaffold_model.labels["microzone-positive"]
    uz_neg = scaffold_model.labels["microzone-negative"]

    S_DCNp = np.intersect1d(S_DCN, uz_pos)
    S_DCNn = np.intersect1d(S_DCN, uz_neg)
    S_IOp = np.intersect1d(S_IO, uz_pos)
    S_IOn = np.intersect1d(S_IO, uz_neg)
    #

    # Prepare adapters
    adapter_forward = scaffold_model.create_adapter("FCN_2019")
    adapter_inverse = scaffold_model.create_adapter("FCN_2019")

    adapter_forward.enable_multi("forward")
    adapter_inverse.enable_multi("reverse")

    adapter_forward.prepare()
    adapter_inverse.prepare()
    #

    # Get NEST populations
    f_IOp = adapter_forward.get_nest_ids(S_IOp)
    f_IOn = adapter_forward.get_nest_ids(S_IOn)

    i_IOp = adapter_inverse.get_nest_ids(S_IOp)
    i_IOn = adapter_inverse.get_nest_ids(S_IOn)

    f_DCNp = adapter_forward.get_nest_ids(S_DCNp)
    f_DCNn = adapter_forward.get_nest_ids(S_DCNn)

    i_DCNp = adapter_forward.get_nest_ids(S_DCNp)
    i_DCNn = adapter_forward.get_nest_ids(S_DCNn)

    f_MF = adapter_forward.get_nest_ids(S_MF)
    i_MF = adapter_inverse.get_nest_ids(S_MF)
    #

    # Define population views
    MF_number = len(f_MF)
    # print("MF_number:", MF_number)  # 89

    planner_pv, cortex_pv = create_cortex(MF_number, prism)

    f_IO_pv = SensoryIO(f_IOp, f_IOn)
    i_IO_pv = MotorIO(i_IOp, i_IOn)

    f_DCN_pv = DirectDCN(f_DCNp, f_DCNn)
    i_DCN_pv = InverseDCN(i_DCNp, i_DCNn)

    f_MF_pv = PopView(f_MF)
    i_MF_pv = PopView(i_MF)
    #

    cortex_pv.connect(f_MF_pv)  # Efference copy
    planner_pv.connect(i_MF_pv)  # Sensory input

    conn_dict = {"rule": "fixed_indegree", "indegree": 1}
    nest.Connect(f_DCNp, cortex_pv.pop, conn_dict, {'weight': 1.0})
    nest.Connect(f_DCNn, cortex_pv.pop, conn_dict, {'weight': -1.0})
    
    forward = Cerebellum(f_MF_pv, f_IO_pv, f_DCN_pv)
    inverse = Cerebellum(i_MF_pv, i_IO_pv, i_DCN_pv)
    
    return Brain(planner_pv, cortex_pv, forward, inverse)

In [7]:
brain = create_brain(0.0)

Importing  NEST...
Locking NEST kernel...
Installing  NEST modules...
cerebmodule
Initializing NEST kernel...
Creating neurons...
Creating dcn_cell_forward...
Creating 6 dcn_cell_forward...
Creating dcn_interneuron_forward...
Creating 6 dcn_interneuron_forward...
Creating golgi_cell_forward...
Creating 51 golgi_cell_forward...
Creating purkinje_cell_forward...
Creating 68 purkinje_cell_forward...
Creating basket_cell_forward...
Creating 99 basket_cell_forward...
Creating stellate_cell_forward...
Creating 200 stellate_cell_forward...
Creating glomerulus_forward...
Creating 1804 glomerulus_forward...
Creating io_cell_forward...
Creating 13628 io_cell_forward...
Creating granule_cell_forward...
Creating 22765 granule_cell_forward...
Creating entities...
Creating mossy_fibers_forward...
Creating 89 mossy_fibers_forward...
Building identifier map...
Creating devices...
Creating device:  poisson_generator
Connecting to 1804 device targets.
Creating device:  poisson_generator
Connecting to 18

In [8]:
planner = brain.planner
cortex = brain.cortex
forward = brain.forward
inverse = brain.inverse

In [9]:
# Get reference x
xs = []

print("Comupting x_0")
for i in range(5):
    nest.Simulate(trial_len)
    x = cortex.integrate(trial_i=i)
    if i >= 1:
         xs.append(x)
    # xs.append(x)

x_0 = np.mean(xs)


print("Comupting x_10")
xs = []

for i in range(5):
    nest.Simulate(trial_len)
    x = cortex.integrate(trial_i=i+5)
    if i >= 1:
         xs.append(x)
    # xs.append(x)

trial_offset = 10

x_10 = np.mean(xs)

get_error = world.get_error_function(x_0, x_10)

Comupting x_0
Comupting x_10


In [10]:
# Get open loop error
xs = []

for i in range(5):
    nest.Simulate(trial_len)

    x = cortex.integrate(trial_i=i+trial_offset)
    if i >= 1:
        xs.append(x)

trial_offset += 5

open_loop_error = get_error(np.mean(xs))
print("Open loop error:", open_loop_error)

Open loop error: 20.70057109098066


In [11]:
def get_weights(pop1, pop2):
    conns = nest.GetConnections(pop1[::50], pop2[::50])
    weights = nest.GetStatus(conns, "weight")
    return weights

In [12]:
sensory_error = open_loop_error
error_history = []

weights_for = []
weights_inv = []

In [13]:
n_trials = 10

for i in range(n_trials):
    forward.io.set_rate(sensory_error)
    inverse.io.set_rate(sensory_error, trial_i=i+trial_offset)

    print("Simulating")
    nest.Simulate(trial_len)
    print()
    print("Trial ", i+1)
    print()

    x_cortex = cortex.integrate(trial_i=i+trial_offset)

    x_dcn = inverse.dcn.integrate(trial_i=i+trial_offset)
    x_sum = x_cortex + x_dcn

    sensory_error = get_error(x_sum)
    error_history.append(sensory_error)
    print("Closed loop error %d:" % i, sensory_error)

    print()
    print("Forward IO: %.1f" % forward.io.get_per_trial_rate())
    print("Forward MF: %.1f" % forward.mf.get_per_trial_rate())
    # print("Forward GR: %.1f" % forward.gr.get_per_trial_rate())
    # print("Forward PC: %.1f" % forward.pc.get_per_trial_rate())
    print("Forward DCN: %.1f" % forward.dcn.get_per_trial_rate())

    # weights = get_weights(forward.gr.pop, forward.pc.pop)
    # weights_for.append(weights)
    # print("Forward PFPC weights:", min(weights), "to", max(weights))

    print()
    print("Inverse IO: %.1f" % inverse.io.get_per_trial_rate())
    print("Inverse MF: %.1f" % inverse.mf.get_per_trial_rate())
    # print("Inverse GR: %.1f" % inverse.gr.get_per_trial_rate())
    # print("Inverse PC: %.1f" % inverse.pc.get_per_trial_rate())
    print("Inverse DCN: %.1f" % inverse.dcn.get_per_trial_rate())

    # weights = get_weights(inverse.gr.pop, inverse.pc.pop)
    # weights_inv.append(weights)
    # print("Inverse PFPC weights:", min(weights), "to", max(weights))

trial_offset += n_trials

Simulating

Trial  1

Closed loop error 0: -483.5581046781375

Forward IO: 10.4
Forward MF: 260.4
Forward DCN: 546.1

Inverse IO: 13.3
Inverse MF: 426.6
Inverse DCN: 546.1
Simulating

Trial  2

Closed loop error 1: -1777.2552076495792

Forward IO: 241.1
Forward MF: 16.0
Forward DCN: 34.4

Inverse IO: 220.1
Inverse MF: 26.7
Inverse DCN: 34.4
Simulating

Trial  3

Closed loop error 2: -21359.98736379178

Forward IO: 888.9
Forward MF: 15.4
Forward DCN: 62.2

Inverse IO: 406.2
Inverse MF: 26.7
Inverse DCN: 62.2
Simulating

Trial  4

Closed loop error 3: -68444.7467835114

Forward IO: 10683.7
Forward MF: 11.5
Forward DCN: 178.3

Inverse IO: 491.2
Inverse MF: 26.7
Inverse DCN: 178.3
Simulating

Trial  5

Closed loop error 4: -90126.45874360095

Forward IO: 34222.7
Forward MF: 10.2
Forward DCN: 276.1

Inverse IO: 495.5
Inverse MF: 26.7
Inverse DCN: 276.1
Simulating

Trial  6

Closed loop error 5: -87973.23525228765

Forward IO: 45061.0
Forward MF: 10.0
Forward DCN: 278.9

Inverse IO: 496.0
In

NESTErrors.C++Exception: ('C++Exception in GetStatus_i: std::bad_alloc', 'C++Exception', <SLILiteral: GetStatus_i>, ': std::bad_alloc')

In [ ]:
brain.planner.plot_spikes()